In [24]:
%load_ext autoreload
%autoreload 2
import dateutil
import json
from pyspark.sql import SQLContext, Row
sqlContext = SQLContext(sc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os
import sys
import inspect

currentdir = os.path.dirname(
    os.path.abspath(inspect.getfile(inspect.currentframe()))
)
parentdir = os.path.dirname(currentdir)
gpparentdir = os.path.dirname(parentdir)
sys.path.insert(0, gpparentdir)

from src.traces.trace_utils import *

In [13]:
def get_hash_function():
    key = name = input("Key ")
    clear_output()
    def hash_function(s):
        code = hmac.new(key.encode('utf-8'), s.encode('utf-8'), hashlib.sha1)
        s = code.hexdigest()
        return s
    return hash_function


def parse_row(line):
    row = line.strip().split('\t')
    if len(row) !=4:
        return None
    
    d = {'ip': row[0],
         'ua': row[1],
         'requests' : parse_requests(row[3])
        }
    return d

def parse_requests(requests):
    ret = []
    for r in requests.split('||'):
        t = r.split('|')
        if len(t) != 3:
            continue
        ret.append({'t': t[0], 'r': t[1], 'p': t[2]})
    ret.sort(key = lambda x: x['t']) # sort by time
    return ret

In [14]:
db = 'traces'
table = 'test'
hive_dir = '/user/hive/warehouse/%s.db' % db
output_dir = '/user/ellery/readers'

In [15]:
hash_function = get_hash_function()

def hash_ip(x):
    x['ip'] = hash_function(x['ip'])
    return x

In [16]:
day = '2016-02-17'
host = 'en.wikipedia.org'

In [17]:
fname = get_file_name(day, host, table)

In [18]:
hive_table = os.path.join(hive_dir, fname)
output_table = os.path.join(output_dir, fname )

In [27]:
trace_rdd = sc.textFile(hive_table) \
                .map(parse_row) \
                .filter(lambda x: x is not None) \
                .map(hash_ip) \
                .map(lambda x: json.dumps(x))         
os.system('hadoop fs -rm -r ' + output_table)
trace_rdd.saveAsTextFile(output_table)